<a href="https://colab.research.google.com/github/AkshayaSobanan/MachineLearning/blob/main/FakeNewsClassifier(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('train.csv',error_bad_lines=False, engine="python")
data.head()

Skipping line 11630: unexpected end of data


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11628 entries, 0 to 11627
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11628 non-null  int64 
 1   title   11312 non-null  object
 2   author  10540 non-null  object
 3   text    11603 non-null  object
 4   label   11628 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 454.3+ KB


In [ ]:
data.describe()

,id,label
count,11628.0000,11628.000000
mean,5813.5000,0.501376
std,3356.8588,0.500020
min,0.0000,0.000000
25%,2906.7500,0.000000
50%,5813.5000,1.000000
75%,8720.2500,1.000000
max,11627.0000,1.000000


In [ ]:
data.isna().sum()

id           0
title      316
author    1088
text        25
label        0
dtype: int64

In [ ]:
data.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
11623,11623,The best halloween costume ever is this guy wh...,Sausage Machine,Next Prev Swipe left/right The best halloween ...,1
11624,11624,What If….,Karl Denninger,"all 30,000 of the “missing” Hillary emails are...",1
11625,11625,"Chaiwali, an Indian Restaurant That Feels Like...",Pete Wells,"Greeting us when we walked into Chaiwali, the ...",0
11626,11626,‘Nothing Short of a Miracle’: No Serious Injur...,Eli Rosenberg,In the ambitious business of New York bridge b...,0


In [ ]:
data.reset_index(inplace=True)

In [ ]:
x=data.drop('label',axis=1)
y=data['label']

In [ ]:
x.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
#Performin Data Preprocessing
#1.Processing title feature by Stemming 
import nltk
import re
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
message=x['title']
porter=PorterStemmer()
def SentenceStemmer(msg):
  review=re.sub('[^a-zA-Z]',' ',str(msg))
  review=review.lower().split()
  review=[porter.stem(word) for word in review if word not in set(stopwords.words('english'))]
  review=' '.join(review)
  return review
x['Clean_Title']=x['title'].apply(SentenceStemmer)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
for x in x['Clean_Title']:
  print(x)
  break

hous dem aid even see comey letter jason chaffetz tweet


In [ ]:
#2.Performing Data Vectorization
#A.Performing One hod representation
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
vocab_size=5000
onehot_list=[one_hot(sentence,vocab_size) for sentence in x['Clean_Title']]
print(onehot_list)


[[772, 1168, 4983, 3124, 4086, 1068, 1287, 4957, 3181, 4493], [1953, 2036, 1298, 369, 1266, 884, 3118], [3849, 2127, 597, 911], [1483, 129, 2346, 3780, 4530, 3080], [443, 1266, 316, 546, 2808, 1072, 1266, 3178, 945, 191], [125, 3950, 1960, 401, 1965, 1073, 941, 3356, 146, 1566, 3103, 3670, 797, 1083, 3118], [2971, 2971, 4559, 1330, 3512, 2822, 3894, 4997, 3865, 1620, 95, 4101], [662, 1375, 3777, 1832, 3645, 27, 3913, 1511, 1090, 3913, 1520], [720, 4464, 3478, 1510, 1073, 4882, 3348, 1374, 3745, 2296, 1090, 3913, 1520], [2773, 4898, 643, 1273, 1431, 523, 1073, 1591, 1090, 3913, 1520], [4874, 2061, 2591, 2724, 2950, 2485, 4778, 4549, 1073, 4928], [3898, 4595, 97, 2231, 350, 2290, 2594, 3503], [1844, 102, 319, 4512, 1174, 4465, 1112, 460, 2303, 2828, 1691], [3780, 961, 4086, 2485, 1073, 1431], [4111, 3717, 2435, 940, 526, 4889, 3503, 541, 1074], [3576, 1054, 4654, 4702, 262, 1020, 420, 1090, 3913, 1520], [2291, 2711, 3730, 3611, 4517, 1090, 3913, 1520], [1116, 2319, 3485, 4015, 1366, 507,

In [ ]:
#B. To even the size of one hot vetor
input_length=20
padd_sequence=pad_sequences(onehot_list,padding='pre',maxlen=input_length)
print(padd_sequence)

[[   0    0    0 ... 4957 3181 4493]
 [   0    0    0 ... 1266  884 3118]
 [   0    0    0 ... 2127  597  911]
 ...
 [   0    0    0 ... 1090 3913 1520]
 [   0    0    0 ... 1090 3913 1520]
 [   0    0    0 ... 1090 3913 1520]]


In [ ]:
padd_sequence[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  772,
       1168, 4983, 3124, 4086, 1068, 1287, 4957, 3181, 4493], dtype=int32)

In [ ]:
#3.Designing the model
from tensorflow.keras.layers import LSTM
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=input_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(padd_sequence),y.shape

(11628, (11628,))

In [ ]:
import numpy as np
X_final=np.array(padd_sequence)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((11628, 20), (11628,))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=40)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
122/122 [==============================] - 6s 36ms/step - loss: 0.4104 - accuracy: 0.8083 - val_loss: 0.2214 - val_accuracy: 0.9112
Epoch 2/10
122/122 [==============================] - 4s 32ms/step - loss: 0.1478 - accuracy: 0.9409 - val_loss: 0.1967 - val_accuracy: 0.9211
Epoch 3/10
122/122 [==============================] - 4s 32ms/step - loss: 0.0947 - accuracy: 0.9659 - val_loss: 0.2037 - val_accuracy: 0.9158
Epoch 4/10
122/122 [==============================] - 4s 31ms/step - loss: 0.0589 - accuracy: 0.9813 - val_loss: 0.2530 - val_accuracy: 0.9179
Epoch 5/10
122/122 [==============================] - 4s 31ms/step - loss: 0.0358 - accuracy: 0.9890 - val_loss: 0.2866 - val_accuracy: 0.9072
Epoch 6/10
122/122 [==============================] - 4s 31ms/step - loss: 0.0195 - accuracy: 0.9954 - val_loss: 0.3700 - val_accuracy: 0.9119
Epoch 7/10
122/122 [==============================] - 4s 32ms/step - loss: 0.0135 - accuracy: 0.9964 - val_loss: 0.4472 - val_accuracy: 0.9033

In [ ]:
#Adding Dropout
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=input_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#Performance Metrics
y_predict=model.predict(x_test)
print(y_predict)

[[0.49886456]
 [0.4986354 ]
 [0.499228  ]
 ...
 [0.4969699 ]
 [0.4972255 ]
 [0.49828196]]


In [ ]:
def bin(i):
  if i>0.5:
    return 1
  else:
    return 0

In [ ]:
y_pred=list(map(bin,y_predict))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1796,  121],
       [1382,  539]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6083897863470558